### Dependencies

In [1]:
using Flux
using BSON
using BSON: @save

### Load pre-trained model

In [2]:
chain = Chain(x -> reshape(x, length(x), 1, 1), MaxPool((1,)), Conv((1,), 1=>16, relu), Conv((1,), 16=>32, relu), x -> reshape(x, :, size(x, 4)), Dense(288, 32, relu), Dense(32, 9), softmax)
BSON.@load "final_weights.BSON" chain

### Start game

In [4]:
print("Welcome! What level do you want to try? Enter 0 for beginner, 1 for advanced: ")
level = parse(Bool, readline())                                                                                                                                                                                       
global grid = fill(0, 9) #I use a 1D grid to implement tic tac toe for efficiency & compatability with the Flux model
global empty = [i for i in 1:9] #list of possible moves
st_first = Bool(rand(0:1))

function displayy(x) 
    m(x) = replace(string.(x), "0"=>"?", "1"=>"X", "-1"=> "O")#replace chars for friendlier UI
    println("-"^7)
    for i in 0:2
        b = m(x)
        println("|", join(b[3*i+1:3*i+3], "|"), "|") #format for printing grid
        println("-"^7)
    end
end

function wincon(x)
    cnt, cnt1 = 1, 1
    validv = false
    for i in 1:3
        # check if there are 3 horizontal/verical collinear points
        if (x[cnt] == x[cnt+1] == x[cnt+2] || x[cnt1] == x[cnt1+6] == x[cnt1+3]) && x[cnt] != 0 && x[cnt1] != 0
            validv = true
            break
        end
        cnt += 3
        cnt1 += 1
    end
    # check for diagonal collinear points
    (validv || (((x[1] == x[5] == x[9]) || (x[7] == x[5] == x[3])) && x[5] != 0)) ? true : false
end

if st_first
    AI_mark = "X"
    println("AI will go first with marker ", AI_mark)
    cnt = 2
else
    mark = "O"
    println("You can go first with marker X")
    cnt = 1
end

counter = 1

while true
    #check win, draw, lose
    global won = wincon(grid)
        
    displayy(grid)
    if counter == 9 && !won
        println("It's a draw!")
        break
    end
        
    if cnt % 2 == 0
        if won
            println("Congrats you win!")
            break
        end
        println("AI's move: ")
        if level
            # chain is the Flux machine learning model
            moves = chain(grid)
            # chain uses softmax to predict probabilities that the next move should be position i (i=1-9)
            # we choose the max value predicted by chain
            move_proposal = findall(x->x==maximum(moves), moves)[1][1]
            if !(move_proposal in empty)
                # AI learns not to make illegal moves (it avoids choosing positions that are already occupied)
                global move = rand(1:length(empty))
            else
                # Exception catching: if AI blunders and chooses an illegal move
                global move = findall(x->x==move_proposal, empty)[1]
            end
        else
            global move = rand(1:length(empty))
        end
        global grid[empty[move]] += 1
        # delete last move from list of possible future moves
        deleteat!(empty, move)
    else
        if won
            println("Sorry game over!")
            break
        end
        print("Pick your move, enter a number from 1 to 9: ")
        global move = parse(Int64, readline())
        global grid[move] -= 1
        deleteat!(empty, findall(i->i==move, empty)[1])
    end
    global cnt += 1
    global counter += 1
end

Welcome! What level do you want to try? Enter 0 for beginner, 1 for advanced: stdin> 1
You can go first with marker X
-------
|?|?|?|
-------
|?|?|?|
-------
|?|?|?|
-------
Pick your move, enter a number from 1 to 9: stdin> 1
-------
|O|?|?|
-------
|?|?|?|
-------
|?|?|?|
-------
AI's move: 
-------
|O|?|?|
-------
|X|?|?|
-------
|?|?|?|
-------
Pick your move, enter a number from 1 to 9: stdin> 9
-------
|O|?|?|
-------
|X|?|?|
-------
|?|?|O|
-------
AI's move: 
-------
|O|X|?|
-------
|X|?|?|
-------
|?|?|O|
-------
Pick your move, enter a number from 1 to 9: stdin> 5
-------
|O|X|?|
-------
|X|O|?|
-------
|?|?|O|
-------
Congrats you win!


### AI Trainer code
I use a deep Neural Network with nonlinear activations to improve accuracy:
chain = Chain(x -> reshape(x, length(x), 1, 1), MaxPool((1,)), Conv((1,), 1=>16, relu), Conv((1,), 16=>32, relu), x -> reshape(x, :, size(x, 4)), Dense(288, 32, relu), Dense(32, 9), softmax)

I didn't have time to train it for too long, but it has already learned blocking (sometimes) and knows what are moves are/are not legal!


In [1]:
function wincon(x)
    cnt2, cnt1 = 1, 1
    validv = false
    for i in 1:3
        if (x[cnt2] == x[cnt2+1] == x[cnt2+2] || x[cnt1] == x[cnt1+6] == x[cnt1+3]) && x[cnt2] != 0 && x[cnt1] != 0
            validv = true
            break
        end
        cnt2 += 3
        cnt1 += 1
    end
    (validv || (((x[1] == x[5] == x[9]) || (x[7] == x[5] == x[3])) && x[5] != 0)) ? true : false
end

global cost, loss = 0,0


using Flux

global chain = Chain(x -> reshape(x, length(x), 1, 1), MaxPool((1,)), Conv((1,), 1=>16, relu), Conv((1,), 16=>32, relu), x -> reshape(x, :, size(x, 4)), Dense(288, 32, relu), Dense(32, 9), softmax)
global opt = ADAM(1e-04)
lossy() = cost/100
gs = gradient(params(chain)) do
    lossy()
end



iter_num = 10000
for j in 1:iter_num
    global cost = 0
    for i in 1:100
        global grid = fill(0, 9)
        global empty = [i for i in 1:9]
        st_first = Bool(rand(0:1))
        st_first ? (global cnt = 2) : (global cnt = 1)
        global counter = 1
        global cost = 0
        global loss = 0
        while true #100 games as minibatch
            global won = wincon(grid)
            if counter == 9 && !won
                global loss = 0
                break
            end
            if cnt % 2 == 0
                if won
                    global loss = 0
                    break
                end
                # random algo move
                global move = rand(1:length(empty))
                global grid[empty[move]] += 1
                deleteat!(empty, move)
            else
                if won
                    global loss += 1
                    break
                end
                moves = chain(grid)
                move_proposal = findall(x->x==maximum(moves), moves)[1][1]
                if !(move_proposal in empty)
                    # AI learns not to make illegal moves
                    global loss += 0.2
                    global move = rand(1:length(empty))
                else
                    global move = findall(x->x==move_proposal, empty)[1]
                    break
                end
                global grid[empty[move]] -= 1
                deleteat!(empty, move)
            end
            global cnt += 1
            global counter += 1
        end
        global cost += loss
        println(cost)
    end
    Flux.Optimise.update!(opt, params(chain), gs)
end
BSON.@save "final_weights.BSON" chain